In [1]:
import os

import pandas as pd
from src.db.connections import SqlServerConnector

### Campos utiles
- Descripción del ticket: content
- Fecha de creación: createdate
- 

In [2]:
# Load data
conn = SqlServerConnector(
    user='Juanmanuel3228 ',
    password=os.getenv('HUBSPOT_PASSWORD'),
    host='datawarehouse-io-eur.database.windows.net',
    port='1433'
)

query = """
SELECT * 
    FROM dbo.Ticket t
    WHERE t.hs_pipeline = '31353452'
    AND t.createdate > '2024-04-15'
    AND t.garantia___tipo_error_postventa is not null;
"""

tickets = conn.query_data(
    query=query,
    database='Juanmanuel2700'
)

In [3]:
tickets.count()

In [4]:
columns_of_interest = [
    'TicketId', 
    'tl_code_ticket',
    'hs_pipeline',
    'hs_pipeline_stage',
    'createdate',
    'garantia___referencia_de_la_incidencia_my_zone'
]

values_of_interest = [
    
    'garantia___tipo_error_postventa',
    's_n',
    'garantia_2___tipo_error_postventa', 
    'garantia_2___s_n', 
    'garantia_3___tipo_error_postventa', 
    'garantia_3___s_n',
    'garantia_4___tipo_error_postventa', 
    'garantia_4___s_n',
    'garantia_5___tipo_error_postventa', 
    'garantia_5___s_n', 
    'garantia_6___tipo_error_postventa', 
    'garantia_6___s_n', 
    'garantia_7___tipo_error_postventa',
    'garantia_7___s_n'
]

tickets_clean = tickets[columns_of_interest + values_of_interest]

tickets_clean.sample(10)

In [5]:
tickets_clean.count()

In [6]:
tickets_clean[tickets_clean['garantia_2___s_n'].isna() & tickets_clean['s_n'].isna() & ~tickets_clean['garantia___tipo_error_postventa'].isna() & ~tickets_clean['garantia_2___tipo_error_postventa'].isna()]

In [7]:
tickets_melted = pd.melt(tickets_clean, id_vars=columns_of_interest, value_vars=values_of_interest).dropna(subset=['value'])

In [17]:
tickets_melted.groupby('variable')['value'].size().sort_values(ascending=False)

In [8]:
sn_columns = [
    's_n',
    'garantia_2___s_n', 
    'garantia_3___s_n',
    'garantia_4___s_n',
    'garantia_5___s_n', 
    'garantia_6___s_n', 
    'garantia_7___s_n'
]

error_type_columns = [
    'garantia___tipo_error_postventa',
    'garantia_2___tipo_error_postventa', 
    'garantia_3___tipo_error_postventa', 
    'garantia_4___tipo_error_postventa', 
    'garantia_5___tipo_error_postventa', 
    'garantia_6___tipo_error_postventa', 
    'garantia_7___tipo_error_postventa',
]

grouped = tickets_melted[tickets_melted['variable'].isin(sn_columns)].groupby(['variable'])['value'].size().sort_values(ascending=False).to_frame()
total = grouped.sum()

grouped['%'] = grouped / total
grouped['cumm %'] = grouped['%'].cumsum()
grouped

In [9]:
tickets_melted[tickets_melted['variable'].isin(sn_columns)]

In [33]:
teste = tickets_melted.copy()

In [34]:
teste

In [35]:
teste[teste['variable'].isin(sn_columns)].groupby('variable')['value'].size().sort_values(ascending=False)

In [36]:
def define_pieza(X):
    if X in ['s_n', 'garantia___tipo_error_postventa']:
        return 1
    elif X in ['garantia_2___s_n', 'garantia_2___tipo_error_postventa']:
        return 2
    elif X in ['garantia_3___s_n', 'garantia_3___tipo_error_postventa']:
        return 3
    elif X in ['garantia_4___s_n', 'garantia_4___tipo_error_postventa']:
        return 4
    elif X in ['garantia_5___s_n', 'garantia_5___tipo_error_postventa']:
        return 5
    elif X in ['garantia_6___s_n', 'garantia_6___tipo_error_postventa']:
        return 6
    elif X in ['garantia_7___s_n', 'garantia_7___tipo_error_postventa']:
        return 7
    else:
        'unknown'

def solve_names(X):
    if X in sn_columns:
        return 'serial_number'
    elif X in error_type_columns:
        return 'error_type'
    return 'unknown'

teste['pieza'] = teste['variable'].apply(define_pieza)
teste['variable2'] = teste['variable'].apply(solve_names)
teste.sample(10)

In [39]:
teste

In [38]:
teste.groupby('pieza')['value'].size().sort_values(ascending=False)

In [22]:
tickets_final = pd.pivot(
    teste,
    index=['TicketId', 'tl_code_ticket', 'hs_pipeline', 'hs_pipeline_stage', 'createdate', 'garantia___referencia_de_la_incidencia_my_zone','pieza'],
    columns='variable',
    values='value'
).reset_index()

tickets_final['total_piezas'] = tickets_final.groupby('TicketId')['TicketId'].transform('count')

In [23]:
tickets_final.groupby('total_piezas')['TicketId'].count()